In [33]:
import sys
sys.path += ["../src"]
import utils
import pandas as pd
import numpy as np
from glob import glob
import matplotlib.pyplot as plt 
import seaborn as sns
from matplotlib.pyplot import subplots as sbp 
from importlib import reload
import jl_vae
# import jl_nflows_geo_coordinates_2 as nfg
# from jl_nflows_geo_coordinates import load_nf as load_dict

from _51_abm_functions import cod_prov_abbrv_df

# Global Spatial Autocorrelation
from spatial_autocorrelation import get_moransI, moransI_scatterplot, hypothesis_testing
# Local Spatial Autocorrelation
from spatial_autocorrelation import get_localMoransI, LISA_scatterplot
from sklearn.preprocessing import MinMaxScaler

"""
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier


from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import precision_recall_curve, average_precision_score
"""

'\nfrom sklearn.linear_model import LogisticRegression\nfrom sklearn.naive_bayes import GaussianNB\nfrom sklearn.neighbors import KNeighborsClassifier\nfrom sklearn.tree import DecisionTreeClassifier\nfrom sklearn.ensemble import RandomForestClassifier\nfrom sklearn.svm import SVC\nfrom sklearn.neural_network import MLPClassifier\n\n\nfrom sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score\nfrom sklearn.metrics import roc_curve, roc_auc_score\nfrom sklearn.metrics import precision_recall_curve, average_precision_score\n'

In [34]:
# transform one-hot encoding to categories
def add_cat_features(df):
    df["energy_class"] = df[[u for u in df.columns if "_energy" in u]].stack().rename("col").reset_index().query("col == 1")["level_1"]
    df["COD_CAT"] = [u[8:] for u in df[[u for u in df.columns if "COD_CAT_" in u]].stack().rename("col").reset_index().query("col == 1")["level_1"]]
    df["anno_costruzione"] = [u[17:] for u in df[[u for u in df.columns if "ANNO_COSTRUZIONE" in u]].stack().rename("col").reset_index().query("col == 1")["level_1"]]
    return df

## Loading data from Jacopo's synthetic data

In [35]:
# import dictionary with data {'hydro_risk', 'census', 'omi_og', 'cap'}
# takes ~25seconds
geo_dict = jl_vae.load_geo_data()

# check which provinces are done
glob(jl_vae.path_pop_synth + f"95sample/pop_samples/*")

['/data/housing/data/intermediate/jl_pop_synth/95sample/pop_samples/synthetic_pop_full_250806priceFE.csv',
 '/data/housing/data/intermediate/jl_pop_synth/95sample/pop_samples/synthetic_pop_full_250703LT.csv',
 '/data/housing/data/intermediate/jl_pop_synth/95sample/pop_samples/synthetic_pop_full_250703FI.csv',
 '/data/housing/data/intermediate/jl_pop_synth/95sample/pop_samples/synthetic_pop_full_250703CE.csv',
 '/data/housing/data/intermediate/jl_pop_synth/95sample/pop_samples/synthetic_pop_full_250703BT.csv',
 '/data/housing/data/intermediate/jl_pop_synth/95sample/pop_samples/synthetic_pop_full_250806priceKR.csv',
 '/data/housing/data/intermediate/jl_pop_synth/95sample/pop_samples/synthetic_pop_full_250703SI.csv',
 '/data/housing/data/intermediate/jl_pop_synth/95sample/pop_samples/synthetic_pop_full_250703CO.csv',
 '/data/housing/data/intermediate/jl_pop_synth/95sample/pop_samples/synthetic_pop_full_250811priceMC.csv',
 '/data/housing/data/intermediate/jl_pop_synth/95sample/pop_samples

In [36]:
import pickle

prov = "AN"

with open(f'/data/housing/data/intermediate/jl_pop_synth/isp_baselines/all_baselines_{prov}.pickle', 'rb') as f:
    all_baselines = pickle.load(f)

In [37]:
all_baselines.keys()

dict_keys(['df_real', 'df_real95', 'df_nfvae', 'df_nfvae95', 'df_ablation', 'df_ablation95', 'df_ipf', 'df_ipf95', 'df_copula_nf', 'df_copula_nf95', 'df_copula_ablation', 'df_copula_ablation95'])

In [38]:
all_baselines['df_real']

,flag_garage,flag_pertinenza,flag_air_conditioning,flag_multi_floor,y,x,log_mq,ANNO_COSTRUZIONE_1500_1965,ANNO_COSTRUZIONE_1965_1985,ANNO_COSTRUZIONE_1985_2005,...,COD_CAT_A_04_05,floor_0.0,floor_1.0,floor_2.0,floor_3.0,floor_Missing,floor_plus_4,log_price,flag_air_conditioning_Missing,flag_multi_floor_Missing
0,1.0,0.0,0.0,0.0,43.615,13.528,4.605170,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,11.813030,1.0,1.0
1,0.0,0.0,0.0,0.0,43.437,13.609,4.304065,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,11.082143,1.0,1.0
2,0.0,0.0,0.0,0.0,43.439,13.610,5.293305,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,11.512925,1.0,1.0
3,1.0,0.0,0.0,0.0,43.658,13.149,4.634729,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,11.813030,1.0,1.0
4,0.0,0.0,0.0,0.0,43.615,13.519,4.532599,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,11.608236,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4247,1.0,0.0,0.0,0.0,43.530,13.441,4.770685,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,11.982929,1.0,1.0
4248,1.0,0.0,0.0,0.0,43.532,13.401,5.068904,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,12.100712,1.0,1.0
4249,1.0,0.0,0.0,0.0,43.534,13.399,5.010635,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,12.427743,1.0,1.0
4250,0.0,0.0,0.0,0.0,43.530,13.441,4.955827,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,11.407565,0.0,1.0


In [39]:
all_baselines['df_real95']

,flag_garage,flag_pertinenza,flag_air_conditioning,flag_multi_floor,y,x,log_mq,ANNO_COSTRUZIONE_1500_1965,ANNO_COSTRUZIONE_1965_1985,ANNO_COSTRUZIONE_1985_2005,...,COD_CAT_A_04_05,floor_0.0,floor_1.0,floor_2.0,floor_3.0,floor_Missing,floor_plus_4,log_price,flag_air_conditioning_Missing,flag_multi_floor_Missing
277,1.0,1.0,0.0,0.0,43.726,13.163,5.220356,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,12.206073,1.0,1.0
1644,0.0,0.0,0.0,0.0,43.606,13.500,4.736198,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,11.472103,1.0,1.0
4001,0.0,0.0,0.0,0.0,43.606,13.495,4.406719,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,11.082143,0.0,1.0
265,1.0,1.0,0.0,0.0,43.626,13.296,5.247024,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,11.695247,1.0,1.0
1756,0.0,1.0,0.0,0.0,43.619,13.528,4.624973,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,11.849398,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1754,0.0,0.0,0.0,0.0,43.608,13.505,4.077537,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,10.645425,1.0,1.0
1419,1.0,0.0,0.0,0.0,43.438,13.642,4.510860,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,12.086726,1.0,1.0
9,1.0,0.0,0.0,0.0,43.524,13.394,4.406719,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,11.736069,1.0,1.0
2335,0.0,1.0,0.0,0.0,43.738,13.189,3.713572,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,11.798104,1.0,1.0


In [40]:
baseline_path = "/data/housing/data/intermediate/jl_pop_synth/pop_samples/baselines"
cod_prov = cod_prov_abbrv_df.query("prov_abbrv == @prov")["COD_PROV"].iloc[0]

df_real = all_baselines['df_real'] 
df_real95 = all_baselines['df_real95']
df_excluded = df_real[~df_real.index.isin(df_real95.index)]

# nf + VAE
df_nfvae = all_baselines['df_nfvae'] 
df_nfvae95 = all_baselines['df_nfvae95'] 
# ablation (only VAE)
df_vae = all_baselines['df_ablation']
df_vae95 = all_baselines['df_ablation95']
# ipf
df_ipf = all_baselines['df_ipf'] 
df_ipf95 = all_baselines['df_ipf95'] 
# copula + nf (95%)
df_copulanf = all_baselines['df_copula_nf']
df_copulanf95 = all_baselines['df_copula_nf95']
# copula (95%)
df_copula = all_baselines['df_copula_ablation']
df_copula95 = all_baselines['df_copula_ablation95']


## fixing data types in syn pop data and get dataframes with the same columns

In [41]:
def ConvertBool2number(df):
    bool_cols = df.select_dtypes(include=bool).columns
    df[bool_cols] = df[bool_cols].astype(float)
    df = df.reset_index(drop=True)
    return df


In [42]:
df_real = ConvertBool2number(df_real)
df_real95 = ConvertBool2number(df_real95)
df_excluded = ConvertBool2number(df_excluded)

# nf + VAE
df_nfvae = ConvertBool2number(df_nfvae)
df_nfvae95 = ConvertBool2number(df_nfvae95)
# ablation (only VAE)
df_vae = ConvertBool2number(df_vae)
df_vae95 = ConvertBool2number(df_vae95)
# ipf
df_ipf = ConvertBool2number(df_ipf)
df_ipf95 = ConvertBool2number(df_ipf95)
# copula + nf
df_copulanf = ConvertBool2number(df_copulanf)
df_copulanf95 = ConvertBool2number(df_copulanf95)
# copula
df_copula = ConvertBool2number(df_copula)
df_copula95 = ConvertBool2number(df_copula95)


Putting all the data in the same space and rescale columns between 0 and 1

In [43]:
df_real95['descr'] = 0 #'real95'
df_excluded['descr'] = 1 #'real_excluded'
# nf + VAE
df_nfvae['descr'] = 2 #'nfvae'
df_nfvae95['descr'] = 3 #'nfvae95'
# ablation (only VAE)
df_vae['descr'] = 4 #'vae'
df_vae95['descr'] = 5 #'vae95'
# ipf
df_ipf['descr'] = 6 #'ipf'
df_ipf95['descr'] = 7 #'ipf95'
# copula + nf
df_copulanf['descr'] = 8 #'copulanf'
df_copulanf95['descr'] = 9 #'copulanf95'
# copula
df_copula['descr'] = 10 #'copula'
df_copula95['descr'] = 11 #'copula95'

df = pd.concat([df_real95,df_excluded,
                df_nfvae,df_nfvae95,
                df_vae,df_vae95,
                df_ipf,df_ipf95,
                df_copulanf,df_copulanf95,
                df_copula,df_copula95])

df = (df-df.min())/(df.max()-df.min())
df

,flag_garage,flag_pertinenza,flag_air_conditioning,flag_multi_floor,y,x,log_mq,ANNO_COSTRUZIONE_1500_1965,ANNO_COSTRUZIONE_1965_1985,ANNO_COSTRUZIONE_1985_2005,...,floor_0.0,floor_1.0,floor_2.0,floor_3.0,floor_Missing,floor_plus_4,log_price,flag_air_conditioning_Missing,flag_multi_floor_Missing,descr
0,1.0,1.0,0.0,0.0,0.951374,0.420027,0.590914,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.563594,1.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.696006,0.815078,0.476087,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.410254,1.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.696006,0.809216,0.397944,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.328785,0.0,1.0,0.0
3,1.0,1.0,0.0,0.0,0.738567,0.575937,0.597239,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.456873,1.0,1.0,0.0
4,0.0,1.0,0.0,0.0,0.723670,0.847901,0.449707,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.489078,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4034,0.0,0.0,0.0,0.0,0.864123,0.357897,0.533947,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.426593,1.0,1.0,1.0
4035,1.0,0.0,0.0,0.0,0.604499,0.847901,0.582124,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.742339,1.0,1.0,1.0
4036,0.0,1.0,0.0,0.0,0.642804,0.693163,0.493826,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.415145,1.0,1.0,1.0
4037,1.0,0.0,0.0,0.0,0.521504,0.427060,0.461569,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.412425,1.0,1.0,1.0


In [44]:
# separate dafaframe again
name_keys = ['real95','real_excluded',
             'nfvae','nfvae95',
             'vae','vae95',
             'ipf','ipf95',
             'copulanf','copulanf95',
             'copula','copula95']

unique_descr = df.descr.unique()
data = dict()
for i in range(len(name_keys)):
    a = df.loc[df.descr == unique_descr[i],:].drop(columns='descr').reset_index(drop=True)
    data[name_keys[i]] = a

In [45]:
for i in data.keys():
    print(i,' num of rows =', len(data[i]),' num of columns=', len(data[i].columns) )


real95  num of rows = 4039  num of columns= 29
real_excluded  num of rows = 213  num of columns= 29
nfvae  num of rows = 4252  num of columns= 29
nfvae95  num of rows = 4039  num of columns= 29
vae  num of rows = 4252  num of columns= 29
vae95  num of rows = 4039  num of columns= 29
ipf  num of rows = 4252  num of columns= 29
ipf95  num of rows = 4039  num of columns= 29
copulanf  num of rows = 4252  num of columns= 29
copulanf95  num of rows = 4039  num of columns= 29
copula  num of rows = 4252  num of columns= 29
copula95  num of rows = 4039  num of columns= 29


## Distance

In [46]:
from sklearn.metrics import pairwise_distances
import gower

def TableDistance(data, control_data, test_data = 'real_excluded',metric='euclidean'):
    
    #df1 = data[test_data] # dataframe containing the data considered in computing the distances
    res = pd.DataFrame(columns=['mean','std','cv','5%','25%','50%','75%','95%'])
    for i in control_data:
        #df2 =  

        # Distance matrix
        if(metric=='euclidean'):
            dists = pairwise_distances(data[test_data].values, data[i].values, metric='euclidean') 
        elif(metric=='norm1'):
            dists = pairwise_distances(data[test_data].values, data[i].values, metric='minkowski', p=1)
        elif(metric == 'gower'):
            dists = gower.gower_matrix(data[test_data], data[i])
        #del df2
        # take the minimum distances
        min_dists = dists.min(axis=1)
        res.loc[i,:] = [np.round(np.mean(min_dists),3), np.round(np.std(min_dists),3), np.round(np.std(min_dists)/np.mean(min_dists),3),
                        np.round(np.quantile(min_dists,0.05),3),np.round(np.quantile(min_dists,0.25),3),np.round(np.median(min_dists),3),
                        np.round(np.quantile(min_dists,0.75),3),np.round(np.quantile(min_dists,0.95),3)]

    return res


In [47]:
control_data = ['nfvae','nfvae95', 'vae','vae95','ipf','ipf95', 'copulanf','copulanf95','copula','copula95']

In [48]:
res_eu = TableDistance(data=data,control_data=control_data,metric='euclidean')
res_eu

,mean,std,cv,5%,25%,50%,75%,95%
nfvae,0.368,0.545,1.48,0.021,0.053,0.114,0.309,1.44
nfvae95,0.656,0.667,1.016,0.025,0.092,0.291,1.415,1.758
vae,0.394,0.523,1.326,0.037,0.081,0.161,0.36,1.435
vae95,0.485,0.558,1.152,0.039,0.09,0.19,0.883,1.456
ipf,0.117,0.128,1.099,0.032,0.056,0.086,0.135,0.258
ipf95,0.175,0.239,1.368,0.033,0.063,0.106,0.2,0.387
copulanf,0.191,0.328,1.717,0.027,0.055,0.091,0.155,1.015
copulanf95,0.182,0.299,1.645,0.033,0.056,0.097,0.161,1.018
copula,0.206,0.311,1.512,0.044,0.077,0.118,0.177,0.809
copula95,0.226,0.343,1.517,0.049,0.08,0.122,0.205,1.036


In [49]:
res_norm1 = TableDistance(data=data,control_data=control_data,metric='norm1')
res_norm1

,mean,std,cv,5%,25%,50%,75%,95%
nfvae,0.612,1.023,1.671,0.034,0.083,0.171,0.507,2.444
nfvae95,1.117,1.284,1.149,0.041,0.152,0.439,2.086,3.559
vae,0.678,1.015,1.497,0.064,0.138,0.269,0.61,2.442
vae95,0.815,1.049,1.286,0.062,0.14,0.307,1.142,2.542
ipf,0.184,0.175,0.95,0.049,0.085,0.14,0.216,0.443
ipf95,0.276,0.367,1.331,0.053,0.104,0.17,0.31,0.695
copulanf,0.321,0.604,1.884,0.043,0.091,0.145,0.267,1.278
copulanf95,0.302,0.531,1.761,0.048,0.089,0.152,0.27,1.31
copula,0.346,0.561,1.618,0.077,0.134,0.187,0.289,1.255
copula95,0.386,0.631,1.636,0.076,0.139,0.2,0.336,1.457


# Try with gower distance

In [50]:
res_gower = TableDistance(data=data,control_data=control_data,metric='gower')
res_gower

,mean,std,cv,5%,25%,50%,75%,95%
nfvae,0.021,0.035,1.652,0.001,0.003,0.006,0.018,0.085
nfvae95,0.04,0.045,1.116,0.002,0.006,0.017,0.073,0.125
vae,0.024,0.035,1.453,0.002,0.005,0.01,0.024,0.086
vae95,0.029,0.037,1.246,0.002,0.005,0.012,0.04,0.09
ipf,0.008,0.007,0.866,0.002,0.004,0.006,0.009,0.018
ipf95,0.011,0.013,1.21,0.002,0.004,0.007,0.013,0.028
copulanf,0.013,0.021,1.688,0.002,0.004,0.006,0.011,0.046
copulanf95,0.012,0.019,1.563,0.002,0.004,0.007,0.011,0.047
copula,0.014,0.02,1.434,0.003,0.006,0.008,0.013,0.049
copula95,0.015,0.022,1.471,0.003,0.006,0.009,0.014,0.054


## Approccio paper neurocomputing

In [51]:
def Distance_x1_x2(x1,x2,metric='euclidean'):
    # Distance matrix
    if(metric=='euclidean'):
        dists = pairwise_distances(x1.values, x2.values, metric='euclidean') 
    elif(metric=='norm1'):
        dists = pairwise_distances(x1.values, x2.values, metric='minkowski', p=1)
    elif(metric == 'gower'):
        dists = gower.gower_matrix(x1, x2)
    #del df2
    # take the minimum distances
    min_dists = dists.min(axis=1)

    return min_dists

def Distance_x1(x1,metric='euclidean'):
    # Distance matrix
    if(metric=='euclidean'):
        dists = pairwise_distances(x1.values, metric='euclidean') 
    elif(metric=='norm1'):
        dists = pairwise_distances(x1.values, metric='minkowski', p=1)
    elif(metric == 'gower'):
        dists = gower.gower_matrix(x1)

    np.fill_diagonal(dists, np.inf) 
    #del df2
    # take the minimum distances
    min_dists = dists.min(axis=1)

    return min_dists

In [53]:
data.keys()

dict_keys(['real95', 'real_excluded', 'nfvae', 'nfvae95', 'vae', 'vae95', 'ipf', 'ipf95', 'copulanf', 'copulanf95', 'copula', 'copula95'])

In [65]:
# function to get the training and testing set
def Get_Train_Test_Sets(data,f=0.8):#,random_state=42):
    res = dict()
    # size of the training set
    Ntrain=int(f*len(data['real95']))
    # size of the testing set
    Ntest = len(data['real_excluded'])

    res['real_train'], _ = Train_Test_Split(data['real95'],Ntrain,Ntest)#,random_state=random_state)
    res['real_test'] = data['real_excluded']

    key_list = ['nfvae', 'nfvae95', 'vae', 'vae95', 'ipf', 'ipf95', 'copulanf', 'copulanf95', 'copula', 'copula95']
    for l in key_list:
        res[l+'_train'], res[l+'_test'] = Train_Test_Split(data[l],Ntrain,Ntest)#,random_state=random_state)

    return res


def Train_Test_Split(df,Ntrain,Ntest):#,random_state=42):
    df_test = df.sample(n=Ntest,replace=False)#,random_state=random_state)
    df_training = df.drop(df_test.index).sample(n=Ntrain,replace = False)#,random_state=random_state)

    df_test = df_test.reset_index(drop=True)
    df_training = df_training.reset_index(drop=True)

    return df_training, df_test


In [56]:
res.keys()

dict_keys(['real_train', 'real_test', 'nfvae_train', 'nfvae_test', 'nfvae95_train', 'nfvae95_test', 'vae_train', 'vae_test', 'vae95_train', 'vae95_test', 'ipf_train', 'ipf_test', 'ipf95_train', 'ipf95_test', 'copulanf_train', 'copulanf_test', 'copulanf95_train', 'copulanf95_test', 'copula_train', 'copula_test', 'copula95_train', 'copula95_test'])

In [66]:
def AdversialAccuracy(df_real,df_syn):
    dts = Distance_x1_x2(df_real,df_syn)
    dtt = Distance_x1(df_real)

    dst = Distance_x1_x2(df_syn, df_real)
    dss = Distance_x1(df_syn)

    AA = 0.5 * (1/len(dts)) *(np.sum(dts>dtt) + np.sum(dst>dss))

    return AA

def NeurocomputingMetrics(df_real_train,df_syn_train,df_real_test,df_syn_test):
    TrLoss = AdversialAccuracy(df_real_train,df_syn_train)
    TeLoss = AdversialAccuracy(df_real_test,df_syn_test)
    PrivacyLoss = TeLoss - TrLoss

    return TrLoss, TeLoss, PrivacyLoss

In [64]:
NeurocomputingMetrics(res['real_train'],res['ipf95_train'],res['real_test'],res['ipf95_test'])

(0.6618693902816466, 0.5446009389671361, -0.11726845131451047)

In [87]:
d_lossTr = {'nfvae':[],
     'nfvae95':[],
     'vae':[],
     'vae95':[],
     'ipf':[],
     'ipf95':[],
     'copulanf':[],
     'copulanf95':[],
     'copula':[],
     'copula95':[]}

d_lossTe = {'nfvae':[],
     'nfvae95':[],
     'vae':[],
     'vae95':[],
     'ipf':[],
     'ipf95':[],
     'copulanf':[],
     'copulanf95':[],
     'copula':[],
     'copula95':[]}

d_lossPr = {'nfvae':[],
     'nfvae95':[],
     'vae':[],
     'vae95':[],
     'ipf':[],
     'ipf95':[],
     'copulanf':[],
     'copulanf95':[],
     'copula':[],
     'copula95':[]}

for i in range(50):
     res = Get_Train_Test_Sets(data)
     for k in d_lossPr.keys():
          a,b,c = NeurocomputingMetrics(res['real_train'],res[k+'_train'],res['real_test'],res[k+'_test'])
          d_lossTr[k].append(a)
          d_lossTe[k].append(b)
          d_lossPr[k].append(c)

          del a,b,c

d_lossPr_mean = dict()
d_lossTe_mean = dict()
d_lossTr_mean = dict()

d_lossPr_std = dict()
d_lossTe_std = dict()
d_lossTr_std = dict()

for k in d_lossTe.keys():
    d_lossPr_mean[k] = np.mean(d_lossPr[k])
    d_lossTe_mean[k] = np.mean(d_lossTe[k])
    d_lossTr_mean[k] = np.mean(d_lossTr[k])
    d_lossPr_std[k] = np.std(d_lossPr[k])
    d_lossTe_std[k] = np.std(d_lossTe[k])
    d_lossTr_std[k] = np.std(d_lossTr[k])

In [88]:
d_lossPr_std

{'nfvae': 0.023950409827408713,
 'nfvae95': 0.013119392922122238,
 'vae': 0.02178799885392094,
 'vae95': 0.020783895413844896,
 'ipf': 0.025218944650374345,
 'ipf95': 0.027882022568567217,
 'copulanf': 0.025604000742645218,
 'copulanf95': 0.025275815490605598,
 'copula': 0.025225742808600335,
 'copula95': 0.02258615855299296}

In [89]:
d_lossTe_std

{'nfvae': 0.02388264341305911,
 'nfvae95': 0.013360829283045415,
 'vae': 0.02170944002496664,
 'vae95': 0.0201358103516031,
 'ipf': 0.024128166963239734,
 'ipf95': 0.026291079812206578,
 'copulanf': 0.025042668958708433,
 'copulanf95': 0.024758482040278456,
 'copula': 0.023735679595038937,
 'copula95': 0.02237836654464053}

In [90]:
d_lossTr_std

{'nfvae': 0.003600904335130985,
 'nfvae95': 0.002157253507484428,
 'vae': 0.002644499618935318,
 'vae95': 0.0025367813825126835,
 'ipf': 0.004402328538952541,
 'ipf95': 0.004497991550536291,
 'copulanf': 0.004507455325524489,
 'copulanf95': 0.0038049832642780552,
 'copula': 0.004649969272288523,
 'copula95': 0.003897185657696732}

In [86]:
summary_results = pd.DataFrame({'model':list(d_lossTr_mean.keys()),
                                'train_loss_mean':list(d_lossTr_mean.values()),
                                'test_loss_mean':list(d_lossTe_mean.values()),
                                'privacy_loss_mean':list(d_lossPr_mean.values())})
summary_results

,model,train_loss_mean,test_loss_mean,privacy_loss_mean
0,nfvae,0.781758,0.634977,-0.146781
1,nfvae95,0.904475,0.791643,-0.112832
2,vae,0.879855,0.717371,-0.162484
3,vae95,0.891232,0.751408,-0.139823
4,ipf,0.654119,0.522394,-0.131725
5,ipf95,0.660616,0.542535,-0.118081
6,copulanf,0.655942,0.597042,-0.058900
7,copulanf95,0.672909,0.607465,-0.065445
8,copula,0.725410,0.626526,-0.098884
9,copula95,0.748106,0.636808,-0.111298


Other tries

In [91]:
np.mean(Distance_x1(data['real95']))

0.1350271687040686

In [98]:
np.mean(Distance_x1_x2(data['nfvae95'],data['real95']))

0.14400214996987643

In [102]:
np.mean(Distance_x1(data['nfvae95']))

0.028751693027323975

In [99]:
np.mean(Distance_x1_x2(data['real95'],data['nfvae95']))

0.7005246244701733

In [96]:
np.mean(Distance_x1_x2(data['real_excluded'],data['nfvae95']))

0.656447043317427

In [103]:
x =Distance_x1_x2(data['real_excluded'],data['nfvae95'])
x.shape

(213,)

In [104]:
y =Distance_x1_x2(data['nfvae95'],data['real_excluded'])
y.shape

(4039,)

In [97]:
np.mean(Distance_x1_x2(data['nfvae95'],data['real_excluded']))

0.294027450556145